# ETL Notebook to Create Database

### Import Dependencies

In [14]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

### Import CSV File and Transform Data

In [2]:
# import billionaires csv file and load to dataframe
billionaire_df = pd.read_csv("static/data/billionaire.csv")
billionaire_df.head()

Name NetWorth        Country         Source  Rank
0                Jeff Bezos   $177 B  United States         Amazon     1
1                 Elon Musk   $151 B  United States  Tesla, SpaceX     2
2  Bernard Arnault & family   $150 B         France           LVMH     3
3                Bill Gates   $124 B  United States      Microsoft     4
4           Mark Zuckerberg    $97 B  United States       Facebook     5

In [3]:
# import coordinates csv file and load to dataframe
coord_df = pd.read_csv("static/data/country_lat_lon.csv")
coord_df.head()

country_code   latitude  longitude               country usa_state_code  \
0           AD  42.546245   1.601554               Andorra             AK   
1           AE  23.424076  53.847818  United Arab Emirates             AL   
2           AF  33.939110  67.709953           Afghanistan             AR   
3           AG  17.060816 -61.796428   Antigua and Barbuda             AZ   
4           AI  18.220554 -63.068615              Anguilla             CA   

   usa_state_latitude  usa_state_longitude   usa_state  
0           63.588753          -154.493062      Alaska  
1           32.318231           -86.902298     Alabama  
2           35.201050           -91.831833    Arkansas  
3           34.048928          -111.093731     Arizona  
4           36.778261          -119.417932  California

In [4]:
# merge billionaires and coordinates dataframes
merged_df = pd.merge(billionaire_df, coord_df, left_on="Country", right_on="country")
merged_df.head()

Name NetWorth        Country              Source  Rank  \
0       Jeff Bezos   $177 B  United States              Amazon     1   
1        Elon Musk   $151 B  United States       Tesla, SpaceX     2   
2       Bill Gates   $124 B  United States           Microsoft     4   
3  Mark Zuckerberg    $97 B  United States            Facebook     5   
4   Warren Buffett    $96 B  United States  Berkshire Hathaway     6   

  country_code  latitude  longitude        country usa_state_code  \
0           US  37.09024 -95.712891  United States            NaN   
1           US  37.09024 -95.712891  United States            NaN   
2           US  37.09024 -95.712891  United States            NaN   
3           US  37.09024 -95.712891  United States            NaN   
4           US  37.09024 -95.712891  United States            NaN   

   usa_state_latitude  usa_state_longitude usa_state  
0                 NaN                  NaN       NaN  
1                 NaN                  NaN       NaN  
2                 NaN                  NaN       NaN  
3                 NaN                  NaN       NaN  
4                 NaN                  NaN       NaN

In [5]:
# transform columns to final dataframe
final_df = merged_df[["Name", "NetWorth", "Country", "Source", "Rank", "latitude", "longitude"]].copy()

final_df = final_df.rename(columns={"Name": "name",
                                    "NetWorth": "networth",
                                    "Country": "country",
                                    "Source": "source",
                                    "Rank": "rank"})

final_df["networth"] = final_df["networth"].map(lambda x: x.lstrip("$").rstrip(" B")).astype(float)

final_df.dtypes

name          object
networth     float64
country       object
source        object
rank           int64
latitude     float64
longitude    float64
dtype: object

In [6]:
#final_df.to_csv("static/data/merged_data.csv", index=False)

In [7]:
# set id as index
final_df["id"] = final_df.index
final_df.set_index("id", inplace=True)
final_df.head()

name  networth        country              source  rank  \
id                                                                       
0        Jeff Bezos     177.0  United States              Amazon     1   
1         Elon Musk     151.0  United States       Tesla, SpaceX     2   
2        Bill Gates     124.0  United States           Microsoft     4   
3   Mark Zuckerberg      97.0  United States            Facebook     5   
4    Warren Buffett      96.0  United States  Berkshire Hathaway     6   

    latitude  longitude  
id                       
0   37.09024 -95.712891  
1   37.09024 -95.712891  
2   37.09024 -95.712891  
3   37.09024 -95.712891  
4   37.09024 -95.712891

### Create postgres Database

In [ ]:
# create connection to postgres billionaires database
connection_string = f"{username}:{password}@localhost:5432/billionaires_db"
engine = create_engine(f"postgresql://{connection_string}")

In [ ]:
# load final dataframe into billionaires table on postgres
final_df.to_sql(name="billionaires", con=engine, if_exists="append", index=True)

In [ ]:
# read data to check that it is there
pd.read_sql_query('select * from billionaires', con=engine).head()